In [1]:
# Importação das Bibliotecas e Parâmetros de Otimização
from functions import *
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.2)
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = '14'
plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.2f' % x) # Tira os números do formato de Notação Científica
np.set_printoptions(suppress=True) # Tira os números do formato de Notação Científica em Numpy Arrays
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning) # Retira Future Warnings

# Spark Session
# from pyspark.sql import SparkSession, Row 
# #from pyspark.sql.functions import * as F
# import pyspark.sql.functions as F
# spark = SparkSession.builder.getOrCreate()

In [2]:
def numero_de_anos_emprego_atual(df):
    df['qt_anos_mesmo_emprego'] = (df['qt_anos_mesmo_emprego'].replace({'< 1 year':0, '1 year':1, '2 years':2, '3 years':3, '4 years':4, '5 years':5, '6 years':6, '7 years':7, '8 years':8, '9 years':9,'10+ years':10}).fillna(0))
    df['qt_anos_mesmo_emprego'] = df['qt_anos_mesmo_emprego'].apply(lambda x:int(x))
    df['qt_anos_mesmo_emprego'] = np.where(df['qt_anos_mesmo_emprego'] <= 3, '3_YEARS', 
                        np.where(df['qt_anos_mesmo_emprego'] <= 6, '6_YEARS',
                        np.where(df['qt_anos_mesmo_emprego'] <= 9, '9_YEARS',
                        '10_YEARS+')))
    return df['qt_anos_mesmo_emprego']

def numero_de_registros_negativos(df):

    df = df[['situacao_do_emprestimo', 'registros_publicos_depreciativos']].copy()
    df[['registros_publicos_depreciativos']] = np.where(df[['registros_publicos_depreciativos']] == 0, 'sem_registros_negativos', 'com_registros_negativos')

    return df['registros_publicos_depreciativos']

def consulta_de_credito_nos_ultimos_6_meses(df):
    df = df[['situacao_do_emprestimo', 'qt_consultas_credito_6meses']].copy()
    df[['qt_consultas_credito_6meses']] = np.where(df[['qt_consultas_credito_6meses']] == 0, 'sem_consultas', 'com_consultas')

    return df['qt_consultas_credito_6meses']

def compromento_de_renda(df): 
    df_aux = df[['faturamento_anual', 'pagamento_mensal', 'valor_emprestimo_solicitado', 'qt_parcelas', 'taxa_de_juros', 'situacao_do_emprestimo']].copy()
    df_aux['qt_parcelas'] = np.where(df_aux['qt_parcelas'] == ' 36 months', 36, 60)
    df_aux['valor_emprestimo_solicitado_com_taxa_de_juros'] = df_aux['pagamento_mensal']*df_aux['qt_parcelas']
    df_aux['pagamento_anual'] = np.where(df_aux['qt_parcelas'] == ' 36 months', df_aux['valor_emprestimo_solicitado_com_taxa_de_juros']/3, df_aux['valor_emprestimo_solicitado_com_taxa_de_juros']/5)
    df_aux['comprometimento_de_renda_anual'] = ((df_aux['pagamento_anual']/df_aux['faturamento_anual'])*100).round(2)
    
    return df_aux['comprometimento_de_renda_anual']

def numero_incidencias_inadimplencia_vencidas_30d(df):
    df_aux = df[['situacao_do_emprestimo', 'inadimplencia_vencida_30dias']].copy()
    df_aux['inadimplencia_vencida_30dias'] = np.where(df_aux[['inadimplencia_vencida_30dias']] == 0, 'sem_inadimplencia_vencida', 'com_inadimplencia_vencida')

    return df_aux['inadimplencia_vencida_30dias']

def n_meses_produto_credito_atual(df):
    df = df.copy()
    df['data_financiamento_emprestimo'] = pd.to_datetime(df['data_financiamento_emprestimo'], format = '%b-%y')
    df['mths_since_data_financiamento_emprestimo'] = round(pd.to_numeric((pd.to_datetime('2023-09-20') - df['data_financiamento_emprestimo'])/np.timedelta64(1, 'M')))
    df['mths_since_data_financiamento_emprestimo'] = df['mths_since_data_financiamento_emprestimo'].fillna(df['mths_since_data_financiamento_emprestimo'].median())
    df['mths_since_data_financiamento_emprestimo'] = np.where(df['mths_since_data_financiamento_emprestimo'] < 0, df['mths_since_data_financiamento_emprestimo'].median(), df['mths_since_data_financiamento_emprestimo'])
    df['mths_since_data_financiamento_emprestimo'] = df['mths_since_data_financiamento_emprestimo'].apply(lambda x:int(x))
    df['data_financiamento_emprestimo'] = df['mths_since_data_financiamento_emprestimo']

    return df['data_financiamento_emprestimo']

def n_meses_primeiro_produto_credito(df):
    df = df.copy()
    df['data_contratacao_primeiro_produto_credito'] = pd.to_datetime(df['data_contratacao_primeiro_produto_credito'], format = '%b-%y')
    df['mths_since_data_contratacao_primeiro_produto_credito'] = round(pd.to_numeric((pd.to_datetime('2023-09-20') - df['data_contratacao_primeiro_produto_credito'])/np.timedelta64(1, 'M')))
    df['mths_since_data_contratacao_primeiro_produto_credito'] = df['mths_since_data_contratacao_primeiro_produto_credito'].fillna(df['mths_since_data_contratacao_primeiro_produto_credito'].median())
    df['mths_since_data_contratacao_primeiro_produto_credito'] = np.where(df['mths_since_data_contratacao_primeiro_produto_credito'] < 0, df['mths_since_data_contratacao_primeiro_produto_credito'].median(), df['mths_since_data_contratacao_primeiro_produto_credito'])
    df['mths_since_data_contratacao_primeiro_produto_credito'] = df['mths_since_data_contratacao_primeiro_produto_credito'].apply(lambda x:int(x))
    df['data_contratacao_primeiro_produto_credito'] = df['mths_since_data_contratacao_primeiro_produto_credito']
    
    return df['data_contratacao_primeiro_produto_credito']

def formato_features_binarias(df):
    df['qt_parcelas'] = np.where(df['qt_parcelas'] == ' 36 months', 0, 1)
    df['inadimplencia_vencida_30dias'] = np.where(df['inadimplencia_vencida_30dias'] == 'sem_inadimplencia_vencida', 0, 1)
    df['tipo_de_concessao_do_credor'] = np.where(df['tipo_de_concessao_do_credor'] == 'f', 0, 1)
    df['plano_de_pagamento'] = np.where(df['plano_de_pagamento'] == 'n', 0, 1)
    df['renda_comprovada'] = np.where(df['renda_comprovada'] == 'Source Verified', 0, 1)
    df['qt_consultas_credito_6meses'] = np.where(df['qt_consultas_credito_6meses'] == 'com_consultas', 0, 1)

    return df

def target_encoder_bad_rate(df, tipo):
    categoricas = ['qt_parcelas', 'grau_de_emprestimo', 'subclasse_de_emprestimo', 'produto_de_credito', 'produto_disponivel_publicamente', 'tipo_de_concessao_do_credor', 'plano_de_pagamento', 'qt_anos_mesmo_emprego', 'status_propriedade_residencial', 'renda_comprovada', 'estado', 'inadimplencia_vencida_30dias', 'registros_publicos_depreciativos', 'qt_consultas_credito_6meses']
    df_aux_2 = df.copy()
    if tipo == 'Criação':
        for cat in categoricas:
            df_aux = df[[f'{cat}', 'situacao_do_emprestimo']].copy()
            good = pd.DataFrame(df_aux.loc[df_aux['situacao_do_emprestimo'] == 0].groupby(f'{cat}', as_index = False)['situacao_do_emprestimo'].count()).rename({'situacao_do_emprestimo':'qt_good'}, axis = 1)
            bad = pd.DataFrame(df_aux.loc[df_aux['situacao_do_emprestimo'] == 1].groupby(f'{cat}', as_index = False)['situacao_do_emprestimo'].count()).rename({'situacao_do_emprestimo':'qt_bad'}, axis = 1)
            df_aux = good.merge(bad, on = f'{cat}', how = 'left')
            df_aux['qt_total'] = df_aux['qt_good'] + df_aux['qt_bad']
            df_aux[f'{cat}_enc'] = ((df_aux['qt_bad']/df_aux['qt_total'])*100).round(2)
            df_aux[f'{cat}_enc'] = df_aux[f'{cat}_enc'].apply(lambda x:float(x))
            df_aux = df_aux[[f'{cat}', f'{cat}_enc']].drop_duplicates().sort_values(by = f'{cat}_enc', ascending = True)
            df_aux.to_csv(f'features/{cat}_enc.csv', index = False)
            df_aux_2 = df_aux_2.merge(df_aux[[f'{cat}', f'{cat}_enc']], on = f'{cat}', how = 'left')
            df_aux_2.drop(f'{cat}', axis = 1, inplace = True)
    else:
        for cat in categoricas:
            ft = pd.read_csv(f'features/{cat}_enc.csv')
            replace_dict = dict(zip(ft[f'{cat}'], ft[f'{cat}_enc']))
            df_aux_2[f'{cat}_enc'] = df_aux_2[f'{cat}'].replace(replace_dict)

    return df_aux_2

In [3]:
df = pd.read_parquet('data/credit_risk_loan_data.parquet').drop(['id', 'url'], axis = 1)
df = df[[
    "loan_status", "acc_now_delinq", "term", "grade", "sub_grade", "purpose", "loan_amnt", "int_rate", "issue_d", "policy_code", "pymnt_plan",
    "initial_list_status", "installment", "emp_title", "emp_length", "home_ownership", "verification_status", "delinq_2yrs", 
    "annual_inc", "addr_state", "tot_cur_bal", "total_rev_hi_lim", "revol_bal", "revol_util", "open_acc", "total_acc", "pub_rec",
    "inq_last_6mths", "earliest_cr_line", "mths_since_last_record", "mths_since_last_major_derog", "mths_since_last_delinq"
]]

df.rename(
    {
        "loan_status": "situacao_do_emprestimo",
        "acc_now_delinq": "qt_contas_inadimplentes_outras_instituicoes",
        "term" : "qt_parcelas",
        "grade" : "grau_de_emprestimo",
        "sub_grade" : "subclasse_de_emprestimo",
        "purpose" : "produto_de_credito",
        "loan_amnt" : "valor_emprestimo_solicitado",
        "int_rate" : "taxa_de_juros",
        "issue_d" : "data_financiamento_emprestimo",
        "policy_code" : "produto_disponivel_publicamente",
        "pymnt_plan" : "plano_de_pagamento",
        "initial_list_status" : "tipo_de_concessao_do_credor",
        "installment" : "pagamento_mensal",
        "emp_title" : "cargo_cliente",
        "emp_length" : "qt_anos_mesmo_emprego",
        "home_ownership" : "status_propriedade_residencial",
        "verification_status" : "renda_comprovada",
        "delinq_2yrs" : "inadimplencia_vencida_30dias",
        "annual_inc" : "faturamento_anual",
        "addr_state" : "estado",
        "tot_cur_bal" : "limite_total_produtos_credito",
        "total_rev_hi_lim" : "limite_total_rotativos",
        "revol_bal" : "limite_rotativos_utilizado",
        "revol_util" : "taxa_utilizacao_limite_rotativos",
        "open_acc" : "qt_produtos_credito_contratados_atualmente",
        "total_acc" : "qt_produtos_credito_contratados_historicamente",
        "pub_rec" : "registros_publicos_depreciativos",
        "inq_last_6mths" : "qt_consultas_credito_6meses",
        "earliest_cr_line" : "data_contratacao_primeiro_produto_credito",
        "mths_since_last_record" : "qt_meses_desde_ultimo_registro_publico",
        "mths_since_last_major_derog" : "qt_meses_classificacao_mais_recente_90dias",
        "mths_since_last_delinq" : "qt_meses_ultima_inadimplencia"
    },
    axis = 1,
    inplace = True
)
df['situacao_do_emprestimo'] = np.where(df['situacao_do_emprestimo'].isin(['Charged Off', 'Default', 'Late (31-120 days)', 'Does not meet the credit policy. Status:Charged Off']) | df['qt_contas_inadimplentes_outras_instituicoes'].fillna(0) != 0, 'BAD', 'GOOD') 
df['situacao_do_emprestimo'] = np.where(df['situacao_do_emprestimo'] == 'BAD', 1, 0)

columns_selected = ['situacao_do_emprestimo', 'qt_parcelas','grau_de_emprestimo','subclasse_de_emprestimo','produto_de_credito', 'inadimplencia_vencida_30dias', 'valor_emprestimo_solicitado','taxa_de_juros','data_financiamento_emprestimo','produto_disponivel_publicamente','plano_de_pagamento','tipo_de_concessao_do_credor','pagamento_mensal','qt_anos_mesmo_emprego','status_propriedade_residencial',
'renda_comprovada','faturamento_anual','estado', 'limite_total_produtos_credito','limite_total_rotativos','limite_rotativos_utilizado','taxa_utilizacao_limite_rotativos','qt_produtos_credito_contratados_atualmente','qt_produtos_credito_contratados_historicamente','registros_publicos_depreciativos','qt_consultas_credito_6meses','data_contratacao_primeiro_produto_credito','qt_meses_desde_ultimo_registro_publico', 'qt_meses_classificacao_mais_recente_90dias',
'qt_meses_ultima_inadimplencia']

df = df[columns_selected]

df_train_raw = separa_treino_teste('situacao_do_emprestimo', df, 0.20)[0]
df_test_raw = separa_treino_teste('situacao_do_emprestimo', df, 0.20)[1] 


df_train_silver = df_train_raw[columns_selected]
df_test_silver= df_test_raw[columns_selected]

df_aux = df_test_silver[:1].copy()

df_aux['qt_anos_mesmo_emprego'] = numero_de_anos_emprego_atual(df_aux)
df_aux['registros_publicos_depreciativos'] = numero_de_registros_negativos(df_aux)
df_aux['qt_consultas_credito_6meses'] = consulta_de_credito_nos_ultimos_6_meses(df_aux)
df_aux['comprometimento_de_renda_anual'] = compromento_de_renda(df_aux)
df_aux['inadimplencia_vencida_30dias'] = numero_incidencias_inadimplencia_vencidas_30d(df_aux)
df_aux['data_financiamento_emprestimo'] = n_meses_produto_credito_atual(df_aux)
df_aux['data_contratacao_primeiro_produto_credito'] = n_meses_primeiro_produto_credito(df_aux)
df_aux = formato_features_binarias(df_aux)
df_aux = target_encoder_bad_rate(df_aux, 'escoragem')

# Feature Selection

features_selected = pd.read_csv('features/features_selected.csv')
features_selected = features_selected.loc[features_selected['importance'] > 0] 
features_selected = list(features_selected['feature'].unique()) + ['situacao_do_emprestimo']
df_aux = df_aux[features_selected].copy()

x = df_aux.drop('situacao_do_emprestimo', axis = 1)
y = df_aux[['situacao_do_emprestimo']]

clf_final = carregar_modelo_pickle('models/clf_final.pkl')

y_pred = clf_final.predict(x)
y_proba= clf_final.predict_proba(x)
teste_threshold = pd.DataFrame({'y_predict':y_pred, 'Proba Good':y_proba[:, 0]})
teste_threshold['y_predict_threshold'] = np.where(teste_threshold['Proba Good'] <= 0.3, 1, 0)
y_pred = teste_threshold['y_predict_threshold'].values

df_aux['Conceder_Credito_Nao_Conceder_Credito'] = y_pred

Modelo carregado de models/clf_final.pkl


In [4]:
df_aux

,comprometimento_de_renda_anual,taxa_utilizacao_limite_rotativos,limite_rotativos_utilizado,limite_total_produtos_credito,faturamento_anual,pagamento_mensal,data_financiamento_emprestimo,taxa_de_juros,limite_total_rotativos,qt_produtos_credito_contratados_historicamente,estado_enc,valor_emprestimo_solicitado,qt_meses_ultima_inadimplencia,qt_produtos_credito_contratados_atualmente,subclasse_de_emprestimo_enc,qt_meses_classificacao_mais_recente_90dias,produto_de_credito_enc,qt_anos_mesmo_emprego_enc,grau_de_emprestimo_enc,qt_meses_desde_ultimo_registro_publico,status_propriedade_residencial_enc,renda_comprovada_enc,qt_consultas_credito_6meses_enc,tipo_de_concessao_do_credor_enc,qt_parcelas_enc,inadimplencia_vencida_30dias_enc,registros_publicos_depreciativos_enc,plano_de_pagamento_enc,situacao_do_emprestimo,Conceder_Credito_Nao_Conceder_Credito
362514,14.27,4.30,1208,6563.00,65000.00,773.01,111,14.99,25100.00,20.00,11.72,32500,NaN,7.00,13.46,NaN,10.31,11.76,11.79,NaN,10.02,11.44,9.54,12.31,14.54,10.95,11.33,11.26,0,0
